In [2]:
!pip install azure-datalake-store

In [3]:
from azure.datalake.store import core, lib, multithread


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
            .master("local[8]") \
            .appName("airflow_app") \
            .config('spark.executor.memory', '16g') \
            .config('spark.driver.memory', '16g') \
            .config('spark.sql.execution.pandas.respectSessionTimeZone', False) \
            .config("spark.driver.maxResultSize", "2048MB") \
            .config("spark.port.maxRetries", "100") \
            .config("spark.sql.execution.arrow.enabled", "true") \
            .getOrCreate()


In [6]:
adlCreds = lib.auth(url_suffix='raizenprd01', resource='https://datalake.azure.net/')

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F2KZ6AY7P to authenticate.


In [7]:
spark.conf.set("fs.adl.oauth2.access.token.provider.type", "RefreshToken")
spark.conf.set("fs.adl.oauth2.client.id", adlCreds.token['client'])
spark.conf.set("fs.adl.oauth2.refresh.token", adlCreds.token['refreshToken'])


In [10]:
df = spark.read.format('csv').option('sep',';').load('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/01_raw/SKU')

In [13]:
df = spark.read.parquet('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/03_primary/vendas_dia')

In [26]:
!pip install plotly-express PyArrow

     |████████████████████████████████| 63.1 MB 3.4 kB/s eta 0:00:01    |█▏                              | 2.3 MB 1.5 MB/s eta 0:00:42     |███████▎                        | 14.4 MB 2.1 MB/s eta 0:00:24     |███████▊                        | 15.2 MB 2.1 MB/s eta 0:00:24     |███████████▍                    | 22.5 MB 2.4 MB/s eta 0:00:17     |███████████▊                    | 23.1 MB 2.4 MB/s eta 0:00:17     |████████████                    | 23.8 MB 2.5 MB/s eta 0:00:16     |████████████▍                   | 24.5 MB 2.5 MB/s eta 0:00:16     |███████████████▏                | 29.9 MB 2.0 MB/s eta 0:00:17     |████████████████                | 31.4 MB 2.8 MB/s eta 0:00:12     |██████████████████▏             | 35.9 MB 2.5 MB/s eta 0:00:12     |███████████████████▍            | 38.2 MB 1.9 MB/s eta 0:00:14     |███████████████████████         | 45.2 MB 2.5 MB/s eta 0:00:08     |███████████████████████▏        | 45.7 MB 2.5 MB/s eta 0:00:07     |████████████████████████▍       | 48.1 MB 2.

In [22]:
import plotly.express as px
from pyspark.sql import functions as F

In [23]:
df2 = df.groupBy("Date").agg({"Volume": "sum"}).withColumn("meio", F.col("sum(Volume)") / 2)

In [24]:
df2.show()

+-------------------+-------------------+--------------------+
|               Date|        sum(Volume)|                meio|
+-------------------+-------------------+--------------------+
|2011-10-10 00:00:00|           6.8392E7|            3.4196E7|
|2011-12-04 00:00:00|           382000.0|            191000.0|
|2012-01-22 00:00:00|           153000.0|             76500.0|
|2012-10-20 00:00:00|            6.005E7|            3.0025E7|
|2013-10-31 00:00:00|           7.8459E7|           3.92295E7|
|2012-07-11 00:00:00|           6.2188E7|            3.1094E7|
|2016-03-11 00:00:00|           8.3718E7|            4.1859E7|
|2016-03-13 00:00:00|           1.0337E7|           5168500.0|
|2014-08-04 00:00:00|           7.7304E7|            3.8652E7|
|2015-04-26 00:00:00|          7587000.0|           3793500.0|
|2015-10-10 00:00:00|           6.9883E7|           3.49415E7|
|2015-07-14 00:00:00|            7.415E7|            3.7075E7|
|2019-06-18 00:00:00|9.069189482519531E7|4.534594741259

In [31]:
dfpandas = df2.orderBy("Date").toPandas()

In [33]:
px.line(dfpandas, x="Date", y=["meio", "sum(Volume)"])

ValueError: All arguments should have the same length. The length of argument `y` is 2, whereas the length of previous arguments ['Date'] is 3225